# Chapter 10. Exploratory Data Analysis in Python

Exploratory Data Analysis, or EDA for short, is the process of cleaning and reviewing data to derive insights such as descriptive statistics and correlation and generate hypotheses for experiments.

EDA results often inform the next steps for the dataset, whether that be generating hypotheses, preparing the data for use in a machine learning model, or even throwing the data out and gathering new data

# 10.1 Getting to know a datset

## Initial exploration

### A first look with ``.head()``

- We'll import the books data from a csv file using pd.read_csv and save it as a DataFrame called "books".
- Taking a look at the top of the DataFrame using the head function, we can see that our data contains columns representing book names, authors, ratings, publishing years, and genres.

```
books = pd.read_csv("books.csv")
books.head()
```

### Gathering more ``.info()``

- pandas also offers a quick way to summarize the number of missing values in each column, the data type of each column, and memory usage using the ``.info`` method.

```
books.info()
```

### Closer look at categorical columns

- A common question about categorical columns in a dataset is how many data points we have in each category.
- We can select the genre column and use the pandas Series method ``.value_counts()`` to find the number of books with each genre.

```
books.value_counts("genre")
```

### ``.describe()`` numerical columns

- Gaining a quick understanding of data included in numerical columns is done with the help of the DataFrame.describe method.
- Calling ``.describe`` on books, we see that it returns the count, mean, and standard deviation of the values in each numerical column (in this case rating and year), along with the min, max, and quartile values.

```
books.describe()
```

### Visualizing numerical data

- Histograms are a classic way to look at the distribution of numerical data by splitting numerical values into discrete bins and visualizing the count of values in each bin.
- To create a histogram, we'll use sns.histplot and pass the books DataFrame as the data argument.

```
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(data=books, x="rating")
plt.show()
```

#### 1. Adjusting bin width

- We can set a bin width of 0.1 using the binwidth keyword argument.
- It will depend on the data's value range

```
sns.histplot(data=books, x="rating", binwidth=0.1)
plt.show()
```

## Data validation

Data validation is an important early step in EDA. We want to understand whether data types and ranges are as expected before we progress too far in our analysis

### Validating data types

- ``.info()`` gives a quick overview of data types included in a dataset along with other information such as the number of non-missing values.
- We can also use the DataFrame ``.dtypes`` attribute if we're only interested in data types.

```
books.info()
books.dtypes
```

### Updating data types

- ``.astype()`` function allows us to change data types without too much effort.
- We redefine the year column by selecting the column and calling the ``.astype()`` method, indicating we'd like to change the column to an integer.
- Then we use the ``.dtypes`` attribute to check that the year column data is now stored as integers

```
books["year"] = books["year"].astype(int)
books.dtypes
```

### Validating categorical data

- We can validate categorical data by comparing values in a column to a list of expected values using ``.isin()``, which can either be applied to a Series as we'll show here or to an entire DataFrame.
- The function returns a Series/DataFrame of the same size and shape as the original but with True and False in place of all values, depending on whether the value from the original Series/DataFrame was included in the list passed to ``.isin()``.

```
books["genre"].isin(["Fiction", "Non Fiction"]) # books whose genres are Fiction and Non Fiction
~books["genre"].isin(["Fiction", "Non Fiction"]) # books that do not belong to the Fiction and Non Fiction genres
```

### Validating numerical data

- We can select and view only the numerical columns in a DataFrame by calling the ``select_dtypes()`` method and passing ``"number"`` as the argument.
- Check the lowest and highest years by using the ``.min()`` and ``.max()`` functions, respectively.
- View a more detailed picture of the distribution of year data using Seaborn's ``boxplot()`` function.
- View the year data grouped by a categorical variable such as genre by setting the ``y`` keyword argument.

```
books.select_dtypes("number").head()
books["year"].min()
books["year"].max()

sns.boxplot(data=books, x="year")
plt.show()

sns.boxplot(data=books, x="year", y="genre")
plt.show()
```

## Data summarization

### Exploring groups of data

- We can explore the characteristics of subsets of data further with the help of the ``.groupby`` function, which groups data by a given category, allowing the user to chain an aggregating function like ``.mean()`` or ``.count()`` to describe the data within each group. 

```
books.groupby("genre").mean()
```

### Aggregating functions

- Sum: `.sum()`
- Count: `.count()`
- Minimum: `.min()`
- Maximum: `.max()`
- Variance: `.var()`
- Standard deviation: `.std()`

### Aggregating ungrouped data

- The ``.agg()`` function, short for aggregate, applis aggreagating functions across a DataFrame.

```
books.agg(["mean", "std"])
```

### Specifying aggregations for columns

- We can even use a dictionary to specify which aggregation functions to apply to which columns.
- The keys in the dictionary are the columns to apply the aggregation, and each value is a list of the specific aggregating functions to apply to that column.

```
books.agg({"rating": ["mean", "std"], "year": ["median"]})
```

### Named summary columns

- By combining ``.agg()`` and ``.groupby()``, we can apply these new exploration skills to grouped data.
- We can create named columns with our desired aggregations by using the ``.agg()`` function and creating named tuples inside it.
- Each named tuple should include a column name followed by the aggregating function to apply to that column.

```
books.groupby("genre").agg(
    mean_rating=("rating", "mean"),
    std_rating=("rating", "std"),
    median_year=("year", "median")
)
```

### Visualizing categorical summaries

- We can display similar information visually using a barplot.
- In Seaborn, bar plots will automatically calculate the mean of a quantitative variable like rating across grouped categorical data.
- Bar plots also show a 95% confidence interval for the mean as a vertical line on the top of each bar.

```
sns.barplot(data=books, x="genre", y="rating")
plt.show()
```

# 10.2 Data cleaning and imputation

## Addressing missing data

### Why is missing data a problem?

1. Affects distributions
    - Missing heights of taller students
2. Less representative of the population
    - Certain groups disproportionately represented
3. Can result in drawing incorrect conlcusions

### Checking for missing values

- We can count the number of missing values per column by chaining the ``.isna()`` and ``.sum()`` methods.
- ``isna`` refers to the fact that missing values are represented as na in DataFrames.

```
print(salaries.isna().sum())
```

### Strategies for addresing missing data

There are various approaches to handle missing data.

- **Drop missing values (if they ammount 5% or less of total values)**
- **Impute mean, median, mode (depends on distirbution and context)**
- **Impute by sub-group (i.e. different experience levels have different median salary)**

#### 1. Dropping missing values

- To calculate our missing values threshold we multiply the length of our DataFrame by five percent
- We can use Boolean indexing to filter for columns with missing values less than or equal to this threshold, storing them as a variable called ``cols_to_drop``.
- We drop missing values by calling ``.dropna()``, passing ``cols_to_drop`` to the subset argument. We set ``inplace=True`` so the DataFrame is updated.

```
threshold = len(salaries) * 0.05
print(threshold)
cols_to_drop = salaries.columns[salaries.isna().sum() <= threshold]
print(cols_to_drop)
salaries.dropna(subset=cols_to_drop, inplace=True)
```

#### 2. Imputing a summary statistic and Checking the remaining values

- We then filter for the remaining columns with missing values.
- To impute the mode for the first three columns, we loop through them and call the ``.fillna()`` method, passing the respective column's ``mode()`` and indexing the first item, which contains the mode, in square brackets.

```
cols_with_missing_values = salaries.columns[salaries.isna().sum() > 0]
print(cols_with_missing_values)
for col in cols_with_missing_values[:-1]: # [:-1] means "all elements of the sequence but the last"
    salaries[col].fillna(salaries[col].mode()[0])
print(salaries.isna().sum())
```

#### 3. Imputing by sub-group

- We use the ``.to_dict()`` method, storing the grouped data as a dictionary.
- i.e. Printing the dictionary returns the median salary for each experience level
- We then impute using the ``.fillna()`` method, providing the Experience column and calling the ``.map()`` method

```
salaries_dict = salaries.groupby("Experience")["Salary_USD"].median().to_dict()
print(salaries_dict)
salaries["Salary_USD"] = salaries["Salary_USD"].fillna(salaries["Experience"].map(salaries_dict))
```

## Converting and analyzing categorical data

### Previewing the data

- We can use the ``select_dtypes()`` method to filter any non-numeric data.
- Chaining ``.head()`` allows us to preview these columns in our salaries DataFrame

```
print(salaries.select_dtypes("object").head())
```

### Examining a column: frequency of values and number of unique values

- Output is truncated by pandas automatically if it is too much information.
- Also, we can count how many unique values there are using pandas ``.nunique()`` method.

```
print(salaries["Designation"].value_counts()) # how many times each unique value appears in the column.
print(salaries["Designation"].nunique()) # gives you the count of unique values
```

### Extracting value from categories

- Current format limits our ability to generate insights
- We can use pandas .str.contains() method, which allows to search for a column for a specific string or multiple strings

```
salaires["Designation"].str.contains("Scientist")
```

### Finding multiple phrases in strings

- Filter for rows containing one or more phrases
- We use the string-dot-contains method again, but this time we include a pipe between our two phrases.
- This will return True if an observation in the Designation column contains (i.e.) Machine Learning or AI, or false if neither of these phrases are present

```
salaires["Designation"].str.contains("Machine learning|AI")
```

- Filter for job titles that start with a specific phrase

```
salaires["Designation"].str.contains("^Data")
```

### Creating a new column with results from finding multiple phrases in strings

#### 1. Creating a list for the values of our new column

- We start by creating a list with the different categories of data roles, which will become the values of a new column in our DataFrame.

```
job_categories = ["Data science", "Data Analytics", "Data engineering", "Machine learning", "Managerial", "Consultant"]
```

#### 2. Create variables with filters

- We then need to create variables containing our filters.

```
data_science = "Data Scientist|NLP"
data_analyst = "Analyst|Analytics"
data_engineer = "Data Engineer|ETL|Architect|Infrastructure"
ml_engineer = "Machine Learning|ML|Big Data|AI"
manager = "Manager|Head|Director|Lead|Principal|Staff"
consultant = "Consultant|Freelance"
```

#### 3. Create a list with range of conditions

- The next step is to create a list with our range of conditions for the ``string.contains`` method.

```
conditions = [
    (salaries["Designations"].str.contains(data_science)),
    (salaries["Designations"].str.contains(data_analyst)),
    (salaries["Designations"].str.contains(data_engineer)),
    (salaries["Designations"].str.contains(ml_engineer)),
    (salaries["Designations"].str.contains(manager)),
    (salaries["Designations"].str.contains(consultant))
]
```

#### 4. Create a new column with results

- Finally, we can create our new Job_Category column by using NumPy's ``np.select()`` function.
    - It takes a list of conditions as the first argument
    - Then a list of arrays to search for the conditions in
    - By using an argument called ``default``, we tell NumPy to assign "Other" when a value in our conditions list is not found.

```
salaries["Job_Category"] = np.select(
    conditions,
    job_categories,
    default="Other"
)
```

#### 5. Previewing job categories

```
print(salaries[["Designation", "Job_Category"]].head())
```

#### 6. Visualizing results frequency

```
sns.countplot(data=salaries, x="Job_Category")
plt.show()
```

## Working with numeric data

### Converting strings to numbers

- To remove commas, we can use the pandas Series .str.replace() method.
- We first pass the characters we want to remove, followed by the characters to replace them with.
- As we don't want to add characters back in, when we update the column we provide an empty string in this part of the method.
- Printing the first five rows of this column, we see the commas have been removed.
- We update the data type to float.

```
pd.Series.str.replace("characters to remove", "characters ot replace them")
salaries["Salary_In_Rupees"] = salaries["Salary_In_Rupees"].str.replace(",", "")
print(salaries["Salary_In_Rupees"].head())
salaries["Salary_In_Rupees"] = salaries["Salary_In_Rupees"].astype(float)
```

### Adding summary statistics into a DataFrame

- Sometimes we might prefer to add summary statistics directly into our DataFrame, rather than creating a summary table.
- Example: Create a new column containing the standard deviation of Salary_USD, where values are conditional based on the Experience column.

```
salaries["std_dev"] = salaries.groupby("Experience")["Salary_USD"].transform(lambda x: x.std())
print(salaries[["Experience", "std_dev"]].value_counts())
```

```
salaries["median_by_comp_size"] = salaries.groupby("Company_Size")["Salary_USD"].transform(lambda x: x.median())
print(salaries[["Company_Size", "median_by_comp_size"]].head())
```

## Handling outliers

### What is an outlier? Why look for them?

- An outlier is an observation that is far away from other data points.
- These are extreme values that may not accurately represent the data.
- Additionally, they can skew the mean and standard deviation.

If we plan to perform statistical tests or build machine learning models, these will often require data that is normally distributed and not skewed!

### Using descriptive statistics

- A starting place for identifying outliers is with the pandas dot-describe method.

```
print(salaries["Salary_USD"].describe())
```

### IQR in boxplots

- These percentiles are included in box plots, like this one showing salaries of data professionals.
- The box contains percentiles, and observations considered to be outliers are represented as diamonds outside of the box.

```
sns.boxplot(data=salaries, y="Salaries_USD")
plt.show()
```

### IQR, identifying threhsolds and oultiers

- We can define an outlier mathematically.
    1. First, we need to know the interquartile range (IQR) which is the difference between the 75th and 25th percentiles.
    2. Once we have the IQR, we can find an upper outlier by looking for values above the sum of the 75th percentile plus one-point-five times the IQR.
    3. Lower outliers have values below the sum of the 25th percentile minus one-point-five times the IQR.
- We can calculate percentiles using the ``Series.quantile`` method.
    1. We pass zero-point-seven-five to find the 75th percentile for salary, then pass zero-point-two-five to get the 25th percentile.
    2. We calculate the IQR by subtracting one from the other.

```
# 75th percentile
seventy_fifty = salaries["Salary_USD"].quantile(0.75)

# 25th percentile
twenty_fifth = salaries["Salary_USD"].quantile(0.25)

# IQR
iqr =  seventy_fifty - twenty_fifth

print(iqr)

# Upper threshold
upper = seventy_fifty + (1.5*iqr)

# Lower treshold
lower = twenty_fifth - (1.5*iqr)

print(upper, lower)
```

### Dropping outliers

- We can remove outliers by modifying the syntax we used to subset our data, filtering for values more than the lower limit and less than the upper limit.

```
no_outliers = salaries[(salaries["Salary_USD"] > lower) & (salaries["Salary_USD"] < upper)]
print(no_outliers["Salary_USD"].describe())
```

# 10.3 Relationships in data

## Patterns over time

### Importing TimeDate data

- Before we can begin to look at potential patterns over time, we need to help pandas understand that data in a given column is in fact date or time data.
- When a CSV file is imported into pandas, date and time data are typically interpreted as strings,
- We can fix that by adding the parse_dates keyword argument to the CSV import and setting it equal to a list of column names that should be interpreted as DateTime data.

```
divorce = pd.read_csv("divorce.csv", parse_dates=["marriage_data"])
divorce.dtypes
```

### Converting to DateTime data

- We may wish to update data types to DateTime data after we import the data.
- This is possible with ``pd.to_datetime()``, which converts the argument passed to it to DateTime data.

```
divorce["marriage_data"] = pd.to_datetime(divorce["marriage_data"])
divorce.dtypes
```

### Create DateTime data

- If a DataFrame has month, day, and year data stored in three different columns, as this one does, we can combine these columns into a single DateTime value by passing them to ``pd.to_datetime()``.
- Note that for this trick to work, columns must be named "month", "day", and "year", but can appear in any order in the DataFrame.

```
divorce["marriage_data"] = pd.to_datetime(divorce[["month", "day", "year"]])
divorce.dtypes
```

### Create DateTime data

- Extract parts of a full date using `dt.month`, `dt.day`, `dt.year`

```
divorce["marriage_month"] = divorce["marriage_date"].dt.month
divorce.dtypes
```

### Visualizing patterns over time

- Line plots are a great way to examine relationships between variables.
- In Seaborn, line plots aggregate y values at each value of x and show the estimated mean and a confidence interval for that estimate.

```
sns.lineplot(data=divorce, x="marriage_month", y="marriage_duration")
plt.show()
```

## Correlation

### Correlation

- Correlation describes the direction of the relationship between two variables as well as its strength.
- Understanding this relationship can help us use variables to predict future outcomes
- However, this highlights an important point about correlations: we must always interpret them within the context of our data

```
divorce.corr()

divorce["divorce_date"].min()
divorce["divorce_date"].max()
```

### Correlation heatmaps

- A heatmap has the benefit of color coding so that strong positive and negative correlations are easier to spot. 
- Setting the ``annot=True`` labels the correlation coefficient inside each cell.

```
sns.heatmap(divorce.corr(), annot=True)
plt.show()
```

### Visualizing relationships

- The Pearson coefficient we've been looking at only describes the linear correlation between variables.
    - Variables can have a strong non-linear relationship and a Pearson correlation coefficient of close to zero.
    - Alternatively, data might have a correlation coefficient indicating a strong linear relationship when another relationship, such as quadratic, is actually a better fit for the data.

```
sns.scatterplot(data=divorce, x="income_man", y="income_woman")
plt.show()
```

- We can take our scatterplots to the next level with Seaborn's pairplot. When passed a DataFrame, pairplot plots all pairwise relationships between numerical variables in one visualization.
- We can limit the number of plotted relationships by setting the ``vars=[list_of_variables]`` argument equal to the variables of interest.

```
sns.pairplot(data=divorce, vars=["income_man", "income_woman", "marriage_duration"])
plt.show()
```

## Factor relationships and distributions

### Exploring categorical relationships

- Categorical variables, or factors, also have relationships.
- They are harder to summarize numerically, so we often rely on visualizations to explore their relationships.

### Kernel Density Estimate (KDE) plots

- Similar to histograms, KDEs allow us to visualize distributions.
- KDEs are considered more interpretable, though, especially when multiple distributions are shown as they are here.
- However, due to the smoothing algorithm used in KDE plots, the curve can include values that don't make sense, so it's important to set good smoothing parameters.
- To fix this, we can use the ``cut`` keyword argument. cut tells Seaborn how far past the minimum and maximum data values the curve should go when smoothing is applied.
    - When we set cut equal to zero, the curve will be limited to values between the minimum and maximum x values

```
sns.kdeplot(data=divorce, x="marriage_duration", hue="education_man", cut=0)
plt.show()
```

### Cummulative KDE plots

- If we're interested in the cumulative distribution function, we can set the ``cumulative=True``.

```
sns.kdeplot(data=divorce, x="marriage_duration", hue="education_man", cut=0, cumulative=True)
plt.show()
```

# 10.4 Turning exploratory analysis into action

## Considerations for categorical data

### Categorical classes and class imbalance

- With categorical data, one of the most important considerations is about the representation of classes, which is another term for labels.
- Class imbalance happens when one class occurs more frequently than others. This can bias results, particularly if this class does not occur more frequently in the population.

### Class frequency

- Number of observations per class

```
print(planes["Destination"].value_counts())
```

### Relative class frequency

- We can use ``value_counts`` method again, but this time set the ``normalize=True`` keyword argument.
- This returns the relative frequencies for each class.

```
print(planes["Destination"].value_counts())
```

### Cross-tabulation

- Enables us to examine the frequency of combinations of classes.
    1. Calling pandas.crosstab function.
    2. Select the column to use as the index for the table
    3. Select names of the columns in the table, and the values will be the count of combined observations.

```
pd.crosstab(planes["Source"], planes["Destination"])
```

### Extending cross-tabulation

- i.e. We can calculate the median price for these routes in our DataFrame, and compare the difference to these expected values.
- We do this by adding two keyword arguments to ``pd.crosstab``.

```
pd.crosstab(planes["Source"], planes["Destination"], values=planes["Price"], aggfunc="median")
```

## Generating new features

Sometimes the format of our data can limit our ability to detect relationships or inhibit the potential performance of machine learning models.

One method to overcome these issues is to generate new features from our data

### Correlation

- Check correlation with a heatmap

```
sns.heatmap(planes.corr(), annot=True)
plt.show()
```

### Data types

- View the data types and establish which ones should be numerical

```
print(planes.dtypes)
```

### Value counts

- We see we need to i.e. remove string characters, and change non-stop to zero, before converting the data type to integer.

```
print(planes["Total_Stops"].value_counts())
```

### Cleaning a column

- We use the ``string.replace`` method to first remove " stops", including the space, so that flights with two, three, or four stops are ready to convert.
- Next we clean flights with one stop.
- Lastly, we change "non-stop" to "0", then set the data type to integer.

```
planes["Total_Stops"] = planes["Total_Stops"].str.replace(" stops", "")
planes["Total_Stops"] = planes["Total_Stops"].str.replace(" stop", "")
planes["Total_Stops"] = planes["Total_Stops"].str.replace("non-stop", "0")
planes["Total_Stops"] = planes["Total_Stops"].astype(int)
```

### Extracting month and weekday

- We know how to extract attributes from datetime values, so we can see if these offer any insights

```
planes["month"] = planes["Date_of_Journey"].dt.month
planes["weekday"] = planes["Date_of_Journey"].dt.weekday
print(planes[["month", "weekday", "Date_of_Journey"]].head())
```

### ``pd.cut()``

- Used to split out numeric data into categories

```
twenty_fifth = planes["Price"].quantile(0.25)
median = planes["Price"].median()
seventy_fifth = planes["Price"].quantile(0.25)
maximum = planes["Price"].max()
labels = ["Economy", "Premium Economy", "Business Class", "First Class"]
bins = [0, twenty_fifth, median, seventy_fifth, maximum]
planes["Price_Category"] = pd.cut(planes["Price"], labels=labels, bins=bins)
print(planes[["Price", "Price_Category"]].head())
sns.countplot(data=planes, x="Airline", hue="Price_Category")

```

## Generating hypothesis

### What is true?

- When performing EDA, the question we should ask is: **how do we know what we are observing is true?**
- To make conclusions regarding relationships, differences, and patterns in our data, we need to use a branch of statistics called **Hypothesis Testing**.
- This involves the following steps before we even start collecting data:
    1. Coming up with a hypothesis, or question
    2. Specifying a statistical test that we will perform in order to reasonably conclude whether the hypothesis was true or false.

### Data Snopping

- The acts of excessive exploratory analysis, the generation of multiple hypotheses, and the execution of multiple statistical tests are collectively known as data snooping, or p-hacking.
- Chances are, if we look at enough data and run enough tests, we will find a significant result.

### Generating hypothesis

- How do we generate hypotheses? We perform some EDA
- Then, we design our experiment. It involves steps such as:
    1. Choosing a sample
    2. Calculating how many data points do we need
    3. Deciding what statistical test to run